## Graph Classification

In [1]:
#  dataset from https://snap.stanford.edu/data/deezer_ego_nets.html
import json
import numpy as np
import pandas as pd

import torch
from torch.nn import Linear
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool

/Users/tabaneslami/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
with open('deezer_ego_nets/deezer_edges.json') as f:
    graphs = json.load(f)


In [3]:
graph_labels = pd.read_csv('deezer_ego_nets/deezer_target.csv')
graph_labels = graph_labels['target'].to_numpy()

In [51]:
def creat_graph(edge_list, label):
    '''
    creates torch geometric graph from 
    edge list and target
    '''
    edge_list = torch.tensor(np.array(edge_list).T)
    label = torch.tensor(label)
    feature_vector = torch.ones((torch.max(edge_list).item()+1),1)
    graph = Data(x=feature_vector, edge_index = edge_list, label = label)
    return graph

In [52]:
all_graphs = []
for i, key in enumerate(graphs.keys()):
    all_graphs.append(creat_graph(graphs[str(key)], graph_labels[i]))
    
train_loader = DataLoader(all_graphs, batch_size=32, shuffle=True)


In [53]:
class graph_classifier(torch.nn.Module):
    def __init__(self, num_features, hidden_channel_1, hidden_channnel2, num_class):
        super(graph_classifier, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_channel_1)
        self.conv2 = GCNConv(hidden_channel_1, hidden_channnel2)
        self.linear = Linear(hidden_channnel2, num_class)
    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = global_mean_pool(x, batch)
        x = self.linear(x)
        return x

In [54]:
model = graph_classifier(1, 20, 10, 2)

In [55]:
for i, data in enumerate(train_loader):
    out = model(data.x, data.edge_index, data.batch)

In [59]:
optimizer = torch.optim.Adam(params=model.parameters())
criterion = torch.nn.CrossEntropyLoss()
for batch in range(100):
    for i, data in enumerate(train_loader):
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        target = data.label
        loss = criterion(out, target)
        loss.backward()
        optimizer.step()
        print(loss)
        
        


tensor(0.6761, grad_fn=<NllLossBackward0>)
tensor(0.7183, grad_fn=<NllLossBackward0>)
tensor(0.7089, grad_fn=<NllLossBackward0>)
tensor(0.6548, grad_fn=<NllLossBackward0>)
tensor(0.7010, grad_fn=<NllLossBackward0>)
tensor(0.7387, grad_fn=<NllLossBackward0>)
tensor(0.6786, grad_fn=<NllLossBackward0>)
tensor(0.6631, grad_fn=<NllLossBackward0>)
tensor(0.6856, grad_fn=<NllLossBackward0>)
tensor(0.6508, grad_fn=<NllLossBackward0>)
tensor(0.6943, grad_fn=<NllLossBackward0>)
tensor(0.6648, grad_fn=<NllLossBackward0>)
tensor(0.7063, grad_fn=<NllLossBackward0>)
tensor(0.6873, grad_fn=<NllLossBackward0>)
tensor(0.7447, grad_fn=<NllLossBackward0>)
tensor(0.6939, grad_fn=<NllLossBackward0>)
tensor(0.6907, grad_fn=<NllLossBackward0>)
tensor(0.6630, grad_fn=<NllLossBackward0>)
tensor(0.6845, grad_fn=<NllLossBackward0>)
tensor(0.7103, grad_fn=<NllLossBackward0>)
tensor(0.6525, grad_fn=<NllLossBackward0>)
tensor(0.6601, grad_fn=<NllLossBackward0>)
tensor(0.6982, grad_fn=<NllLossBackward0>)
tensor(0.69

KeyboardInterrupt: 